# **MIP1_Run2**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """You are a city planner, looking to open facilities at some locations. We have a set of customers and a set of possible locations for opening facilities. Each potential location for establishing a facility incurs a fixed annual activation cost, which must be paid if the facility is used, regardless of the service volume it handles. Furthermore, this service volume at each facility is capped at a maximum annual limit. Additionally, there are transportation costs associated with servicing each customer from each facility.
The goal is to minimize the overall costs, which include both the fixed activation costs for any opened facilities and the transportation costs for servicing customers. This must be done while making sure that each customer’s demand is met, each facility does not exceed its maximum service volume, and customers can of course only be serviced by a facility that is opened.
Please formulate this as a mathematical optimization model."""

In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [6]:
print(response1.choices[0].message.content)

Variables:
- Binary variable x[i] for each potential location i (x[i] = 1 if a facility is built at location i, and 0 otherwise).
- Binary variable y[i][j] for each customer j and location i (y[i][j] = 1 if customer j is served by the facility at location i, and 0 otherwise).
- Continuous variable v[i] representing the service volume at facility i.
- Constant c[i] representing the fixed annual activation cost at location i.
- Constant t[i][j] representing the transportation cost of servicing customer j from facility i.
- Constant d[j] representing the demand of customer j.
- Constant max_v[i] representing the maximum annual service volume of facility i.


In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [8]:
print(response2.choices[0].message.content)

Objective Function:

Minimize (Summation over all i (c[i]*x[i]) +  Summation over all i and j (t[i][j]*y[i][j]))


In [9]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [10]:
print(response3.choices[0].message.content)

Constraints:

1. Customer's Demand: For each customer j, the sum of y[i][j] over all locations i must be equal to 1. This ensures that each customer's demand is met.

   Summation over all i (y[i][j]) = 1 for all j .

2. Facility Service Volume Limit: For each facility i, the sum of d[j]*y[i][j] over all customers j must not exceed the maximum service volume of the facility.

   Summation over all j (d[j]*y[i][j]) <= max_v[i]*x[i] for all i .

3. Customer can only be served by an opened facility: For each facility i and customer j, y[i][j] must be less than or equal to x[i].

   y[i][j] <= x[i] for all i,j.

4. Binary Constraints: For each facility i and customer j, y[i][j] and x[i] are binary i.e., y[i][j] ∈ {0,1} and x[i] ∈ {0,1}.

5. Non-negativity Constraint: v[i] >= 0 for all i. This implies that the service volume at any facility cannot be negative.


### **Generate Pyomo Code**

In [11]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [12]:
Markdown(response4.choices[0].message.content)

Here is a Python code implementing the optimization problem using Pyomo:

```python
# Required Pyomo Package
from pyomo.environ import *

# Complete set of facility locations and customers
locations = ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix']    # [i]
customers = ['Customer1', 'Customer2', 'Customer3', 'Customer4', 'Customer5']   # [j]

# Sample constants
c = {'New York': 10, 'Los Angeles': 12, 'Chicago': 15, 'Houston': 17, 'Phoenix': 13} # activation costs
t = {('New York', 'Customer1'): 5, ('New York', 'Customer2'): 4,
    ...}    # define all t[i][j] constants, the transportation costs
d = {'Customer1': 3, 'Customer2': 4, 'Customer3': 8, 'Customer4': 6, 'Customer5': 2} # demand of customers
max_v = {'New York': 15, 'Los Angeles': 14, 'Chicago': 16, 'Houston': 15, 'Phoenix': 16} # maximum annual service volume

# Create a concrete model
model = ConcreteModel()

# Define sets over locations(i) and customers(j)
model.locations = Set(initialize=locations)
model.customers = Set(initialize=customers)

# Define the decision variables
model.x = Var(model.locations, domain=Binary)
model.y = Var(model.locations, model.customers, domain=Binary)
model.v = Var(model.locations, domain=NonNegativeReals)

# Define the objective function
model.obj = Objective(
    expr=sum(c[i]*model.x[i] for i in model.locations) + sum(t[i,j]*model.y[i,j] for i in model.locations for j in model.customers),
    sense=minimize)

# Define the constraints
model.demand = Constraint(model.customers, rule=lambda model, j : sum(model.y[i,j] for i in model.locations) == 1)
model.volume = Constraint(model.locations, rule=lambda model, i : sum(d[j]*model.y[i,j] for j in model.customers) <= max_v[i]*model.x[i])
model.assign = Constraint(model.locations, model.customers, rule=lambda model, i, j : model.y[i,j] <= model.x[i])

# Solves the model
solver = SolverFactory('glpk')
solver.solve(model)

# Prints the output
for i in model.locations:
    print('Facility in %s: %d' % (i, model.x[i].value))
    for j in model.customers:
        if model.y[i,j].value ==1:
            print('Customer %s served in %s' %(j,i))
```

Note:
- Please fill in your own data for t, which I used '…' to replace.
- This code uses the GLPK solver, which is free and open source. However, you may replace it with other solvers if you want.

### **Run the code Generated by GPT4**

In [15]:
from pyomo.environ import *

# Complete set of facility locations and customers
locations = ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix']    # [i]
customers = ['Customer1', 'Customer2', 'Customer3', 'Customer4', 'Customer5']   # [j]

# Sample constants
c = {'New York': 10, 'Los Angeles': 12, 'Chicago': 15, 'Houston': 17, 'Phoenix': 13} # activation costs
t = {('New York', 'Customer1'): 5, ('New York', 'Customer2'): 4,
    ...}    # define all t[i][j] constants, the transportation costs
d = {'Customer1': 3, 'Customer2': 4, 'Customer3': 8, 'Customer4': 6, 'Customer5': 2} # demand of customers
max_v = {'New York': 15, 'Los Angeles': 14, 'Chicago': 16, 'Houston': 15, 'Phoenix': 16} # maximum annual service volume

# Create a concrete model
model = ConcreteModel()

# Define sets over locations(i) and customers(j)
model.locations = Set(initialize=locations)
model.customers = Set(initialize=customers)

# Define the decision variables
model.x = Var(model.locations, domain=Binary)
model.y = Var(model.locations, model.customers, domain=Binary)
model.v = Var(model.locations, domain=NonNegativeReals)

# Define the objective function
model.obj = Objective(
    expr=sum(c[i]*model.x[i] for i in model.locations) + sum(t[i,j]*model.y[i,j] for i in model.locations for j in model.customers),
    sense=minimize)

# Define the constraints
model.demand = Constraint(model.customers, rule=lambda model, j : sum(model.y[i,j] for i in model.locations) == 1)
model.volume = Constraint(model.locations, rule=lambda model, i : sum(d[j]*model.y[i,j] for j in model.customers) <= max_v[i]*model.x[i])
model.assign = Constraint(model.locations, model.customers, rule=lambda model, i, j : model.y[i,j] <= model.x[i])

# Solves the model
solver = SolverFactory('glpk')
solver.solve(model)

# Prints the output
for i in model.locations:
    print('Facility in %s: %d' % (i, model.x[i].value))
    for j in model.customers:
        if model.y[i,j].value ==1:
            print('Customer %s served in %s' %(j,i))

SyntaxError: ':' expected after dictionary key (<ipython-input-15-cc6980a9df7a>, line 10)

In [17]:
from pyomo.environ import *

# Complete set of facility locations and customers
locations = range(1,4)    # [i]
customers = range(1,6)   # [j]

# Sample constants
c = {1: 1000, 2: 1000, 3: 1000} # activation costs
t = {
    (1, 1): 4,  (2, 1): 6,  (3, 1): 9,
    (1, 2): 5,  (2, 2): 4,  (3, 2): 7,
    (1, 3): 6,  (2, 3): 3,  (3, 3): 4,
    (1,4): 8,  (2,4): 5,  (3,4): 3,
    (1,5): 10, (2,5): 8,  (3,5): 4,
}    # define all t[i][j] constants, the transportation costs
d = {1: 80, 2: 270, 3: 250, 4: 160, 5: 180}  # demand of customers
max_v = {1: 500, 2: 500, 3: 500}   # maximum annual service volume

# Create a concrete model
model = ConcreteModel()

# Define sets over locations(i) and customers(j)
model.locations = Set(initialize=locations)
model.customers = Set(initialize=customers)

# Define the decision variables
model.x = Var(model.locations, domain=Binary)
model.y = Var(model.locations, model.customers, domain=Binary)
model.v = Var(model.locations, domain=NonNegativeReals)

# Define the objective function
model.obj = Objective(
    expr=sum(c[i]*model.x[i] for i in model.locations) + sum(t[i,j]*model.y[i,j] for i in model.locations for j in model.customers),
    sense=minimize)

# Define the constraints
model.demand = Constraint(model.customers, rule=lambda model, j : sum(model.y[i,j] for i in model.locations) == 1)
model.volume = Constraint(model.locations, rule=lambda model, i : sum(d[j]*model.y[i,j] for j in model.customers) <= max_v[i]*model.x[i])
model.assign = Constraint(model.locations, model.customers, rule=lambda model, i, j : model.y[i,j] <= model.x[i])

# Solves the model
solver = SolverFactory('glpk')
solver.solve(model)

# Prints the output
for i in model.locations:
    print('Facility in %s: %d' % (i, model.x[i].value))
    for j in model.customers:
        if model.y[i,j].value ==1:
            print('Customer %s served in %s' %(j,i))
print(model.obj())

Facility in 1: 0
Facility in 2: 1
Customer 1 served in 2
Customer 3 served in 2
Customer 4 served in 2
Facility in 3: 1
Customer 2 served in 3
Customer 5 served in 3
2025.0


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**